In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#This file is contains the score of differnt teams
output = pd.read_csv('output.csv')

# Convert all the entities to lower case
output['name'] = output['name'].apply(lambda x: x.lower())

# remove all punctuations and spaces, now concatenate the string
output['name'] = output['name'].str.replace(r'\W+', '').astype('str')

#unnecessary column
output.drop(columns=['score2'],inplace = True)

#based on the value of the columns divide into different classes.
output['isEff'] = np.where(output.score1 > 50, np.where(output.score1 < 70, 'Good','Very Good') , np.where(output.score1 < 50, 'Very Bad',' Bad'))
output['isEff'] = np.where(output.score1 > 55, 'Yes' , 'No')

output.team_id = output.team_id.astype(str)
output.isEff

0     Yes
1     Yes
2     Yes
3     Yes
4     Yes
5      No
6      No
7      No
8      No
9      No
10     No
11     No
12    Yes
13    Yes
14    Yes
15    Yes
16    Yes
17    Yes
18     No
19     No
20     No
21     No
22     No
23     No
24    Yes
25    Yes
26     No
27    Yes
28     No
29     No
     ... 
53    Yes
54     No
55     No
56     No
57    Yes
58     No
59     No
60     No
61     No
62    Yes
63     No
64    Yes
65     No
66    Yes
67     No
68    Yes
69    Yes
70     No
71     No
72     No
73     No
74     No
75     No
76    Yes
77    Yes
78    Yes
79    Yes
80    Yes
81    Yes
82    Yes
Name: isEff, Length: 83, dtype: object

In [3]:
# Create dictionaries
nameMap = dict(zip(output.name,output.isEff))
idMap = dict(zip(output.team_id,output.isEff))
idToNameMap = dict(zip(output.team_id,output.name))

#nameMap = output.groupby('name')['isEff'].apply(list).to_dict()
#idMap = output.groupby('team_id')['isEff'].apply(list).to_dict()
#IdToNameMap = output.groupby('team_id')['name'].apply(list).to_dict()


In [4]:
# Read Data 
data1 = pd.read_csv('TeamPerformance1.csv')
data2 = pd.read_csv('TeamPerformance2.csv')

data1 = data1.rename(columns = {'Team Number':'teamName',
                                'Email address':'email',
                                'Member Contribution:':'equalContribution',
                                'Extrovert / Introvert:':'EI',
                                'Sensation / Intuition:':'SI',
                                'Thinking / Feeling:':'TF',
                                'Judgement / Perception:':'JP',
                                'Do you wish to Work with the same team again?':'workAgain',
                                'To what extent can your project be improved?':'howMuchImproved',
                               'Have you worked with your team earlier in any other project?':'workedEarlier',
                                'Would you like to work with your team again?':'workAgain',
                               'Difficulty Level of your project :':'Difficulty',
                                'Can your project be improved further?':'Improvement',
                               'What kind of improvement do you wish to make?':'whatImprovement',
                                'Team members have made equal contributions to the project:':'equalContribution',
                               "Team members appreciate one another's unique capability":'appreciation',
                                "Team members are cooperative (agree with one another's opinions, consistently help each other and are effective listeners):":'cooperation',
                               'It is easy to talk openly to all members of your group':'freedom',
                                'Additional comments':'comments',
                               'Are you a Team Leader?':'TeamLead',
                               'Domain of your Project:':'Domain'})


data2 = data2.rename(columns = {'Team Number':'teamName',
                                'Personality type (Example: ENTJ-A)':'Personality',
                                'Email address':'email',
                                'Extrovert / Introvert:':'EI',
                                'Sensation / Intuition:':'SI',
                                'Thinking / Feeling:':'TF',
                                'Judgement / Perception:':'JP',
                                'Do you wish to Work with the same team again?':'workAgain',
                                'To what extent can your project be improved?':'howMuchImproved',
                               'Have you worked with your team earlier in any other project?':'workedEarlier',
                                'Would you like to work with your team again?':'workAgain',
                               'Difficulty Level of your project :':'Difficulty',
                                'Can your project be improved further?':'Improvement',
                               'What kind of improvement do you wish to make?':'whatImprovement',
                                'Team members have made equal contributions to the project:':'equalContribution',
                               "Team members appreciate one another's unique capability":'appreciation',
                                "Team members are cooperative (agree with one another's opinions, consistently help each other and are effective listeners):":'cooperation',
                               'It is easy to talk openly to all members of your group':'freedom',
                                'Additional comments':'comments'})


# Assuming 'No' if the questions are not answered  
data1['Improvement'] = np.where(data1.howMuchImproved != 'Yes', 'No', data1.howMuchImproved)

# Removing the NaN values
data1 = data1[data1.Personality.isnull() == False]

# Concatenating the the two data samples
data = pd.concat([data1,data2],sort = False)

# Drop unnecessary columns
data.drop(['Team size','whatImprovement','Timestamp','USN/SSN','comments','email','Domain','TeamLead','howMuchImproved'],axis = 1, inplace = True)

# Adding the 'Temperament' Attribute
data['Temperament'] = np.where(data.SI == 'Sensation',np.where(data.JP == 'Judgement','Gaurdian','Artisans'), np.where(data.TF == 'Feeling','Idealists','Rationals'))

In [5]:
def teamNameProcessing(data):
    data['teamName'] = data['teamName'].astype('str').apply(lambda x: x.lower())
    data['teamName'] = data['teamName'].str.replace(r'\W+','').astype('str')
    

def convertToYesNo(data,attr,val):
    data[attr] = np.where(data[attr] >= val, 'Yes', 'No')

    

teamNameProcessing(data)
    
convertToYesNo(data,'Difficulty',4)
convertToYesNo(data,'equalContribution',4)
convertToYesNo(data,'appreciation',4)
convertToYesNo(data,'cooperation',4)
convertToYesNo(data,'freedom',4)

In [6]:
# Create a new column isEff
def isEff(data):
    data['isEff'] = data['teamName'].map(nameMap)
    data['teamName'] = np.where(data['teamName'].str.contains(pat = '^[0-9].*'),data['teamName'].map(idToNameMap), data['teamName'] )
    data['isEff'] = np.where(pd.isnull(data.isEff) ,data['teamName'].map(nameMap),data.isEff)
    data['isEff'] = np.where(pd.isnull(data.isEff) ,data['teamName'].map(idMap),data.isEff)
    #print(data[['teamName','isEff']])
    return data




In [7]:
def femalePerc(x,val1,val2):
    c1 = 0
    c2 = 0
    for a in x:    
        if a == val1:
            c1=c1+1
        elif a == val2:
            c2=c2+1
            
    ratio = round(c1/(c1+c2),2)*100
    return ratio

def convertToTeamData(data,newAttr,oldAttr,val1,val2):
    data[newAttr] = data[oldAttr].str.split(',').apply(lambda x:sorted(x))
    data[newAttr] =  data[newAttr].apply(femalePerc, args = (val1,val2))
    return data



In [8]:
teams = data.groupby('teamName').agg(','.join).reset_index()

teams['size'] = teams['Gender'].str.split(',').str.len()
teams = isEff(teams)
teams = convertToTeamData(teams,'femalePercent','Gender','Female','Male')    
teams = convertToTeamData(teams,'IntrovertPercent','EI','Introvert','Extrovert')  
teams = convertToTeamData(teams,'IntuitionPercent','SI','Intuition','Sensation')
teams = convertToTeamData(teams,'ThinkingPercent','TF','Thinking','Feeling')
teams = convertToTeamData(teams,'JudgementPercent','JP','Judgement','Perception')  
teams = convertToTeamData(teams,'workedEarlierPercent','workedEarlier','Yes','No')  
teams = convertToTeamData(teams,'workAgainPercent','workAgain','Yes','No') 
teams = convertToTeamData(teams,'workAgainPercent','workAgain','Yes','No') 
teams = convertToTeamData(teams,'DifficultyPercent','Difficulty','Yes','No') 
teams = convertToTeamData(teams,'equalContributionPercent','equalContribution','Yes','No')
teams = convertToTeamData(teams,'cooperationPercent','cooperation','Yes','No')
teams = convertToTeamData(teams,'freedomPercent','freedom','Yes','No')
teams = convertToTeamData(teams,'ImprovementPercent','Improvement','Yes','No')



teams.to_csv('FinalData.csv')
print(teams.isEff)


0     Yes
1      No
2      No
3     Yes
4     Yes
5     Yes
6     Yes
7     Yes
8     Yes
9     Yes
10     No
11     No
12     No
13    Yes
14     No
15     No
16    Yes
17     No
18    Yes
19    Yes
20     No
21    Yes
22     No
23    Yes
24    Yes
25     No
26     No
27    Yes
28     No
29    Yes
30    Yes
31    Yes
32     No
33     No
34    Yes
35    Yes
36    Yes
37    Yes
38    Yes
39     No
40     No
41    Yes
42     No
43    Yes
44     No
45     No
46    Yes
47     No
48     No
49     No
50    Yes
51    Yes
52    Yes
53     No
54    Yes
55     No
56     No
57     No
58    Yes
59     No
Name: isEff, dtype: object


In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


y = teams['isEff']
print(y.value_counts())
X = teams.drop(['isEff','teamName','Gender', 'Personality', 'EI', 'SI', 'TF', 'JP',
       'workedEarlier', 'workAgain', 'Difficulty', 'equalContribution',
       'appreciation', 'cooperation', 'freedom', 'Improvement','Temperament'],axis = 1)

train_x,test_x,train_y,test_y = train_test_split(X,y)
# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(train_x, train_y)

# Actual class predictions
rf_predictions = model.predict(test_x)
# Probabilities for each class
rf_probs = model.predict_proba(test_x)



/home/nimisha/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Yes    32
No     28
Name: isEff, dtype: int64


import json,csv
file = open('../Data/teams.json')
data = json.load(file)
output = open('../Data/output.csv','w')
op = csv.writer(output)
f = 0
for d in data:
    if f == 0:
        header = d.keys()
        head = []
        a = ''
        for h in header:
            head.append(h)
        
        header  = ','.join(head)
        op.writerow(head)
        
        print(a)
        f += 1
    else:
        
        row = d.values()
        #print(type(row))
        #op.writerow(row)
file.close()
row = ['team_id','name','room','score1','score2']
with open('output.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(row)
    rows = []
    for d in data:
        row = []
        row.append(d['team_id'])
        row.append(d['name'])
        row.append(d['room'])
        info = d['scores'][0]
        if len(info) == 0:
            print(d)
            data.remove(d)
        elif len(info) != 3:
            for i in range(len(info)):
                score1 = info[i]['score']
                row.append(score1)
        rows.append(row)
        writer.writerow(row)
csvFile.close()
